Primero leemos los ficheros iniciales de entrada.

In [117]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from statistics import mode
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score


In [118]:
pcosData = pd.read_csv('./datos/pcos.csv', skiprows = 1, header = None,
                           names=['Age (yrs)', 'Weight (Kg)', 'Height(Cm)', 'BMI', 'Blood Group', 'Pulse rate(bpm)',
                                   'RR (breaths/min)', 'Hb(g/dl)', 'Cycle(R/I)', 'Cycle length(days)', 'Marriage Status (Yrs)',
                                     'Pregnant(Y/N)', 'No. of abortions', 'I beta-HCG(mIU/mL)', 'FSH(mIU/mL)', 'LH(mIU/mL)', 'FSH/LH', 'Hip(inch)',
                                       'Waist(inch)', 'Waist:Hip Ratio', 'TSH (mIU/L)', 'PRL(ng/mL)', 'Vit D3 (ng/mL)', 'PRG(ng/mL)', 'RBS(mg/dl)', 'Weight gain(Y/N)', 'hair growth(Y/N)',
                                         'Skin darkening (Y/N)', 'Hair loss(Y/N)', 'Pimples(Y/N)', 'Fast food (Y/N)', 'Reg.Exercise(Y/N)', 'BP _Systolic (mmHg)', 'BP _Diastolic (mmHg)', 'Follicle No. (L)',
                                           'Follicle No. (R)', 'Avg. F size (L) (mm)', 'Avg. F size (R) (mm)', 'Endometrium (mm)', 'PCOS (Y/N)'])


titanicData = pd.read_csv('./datos/titanic.csv', skiprows = 1, header = None,
                              names=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Initial', 'Age_band', 'Family_Size', 'Alone', 'Fare_cat', 'Deck', 'Title', 'Is_Married', 'Survived'])


Definimos la funcion para estandarizar los ficheros.

In [119]:

def dataStandarization (data, dataName, excludedColumns):
    if not isinstance (dataName, str):
        print("El argumento dataName debe ser una cadena de carateres.")
    
    elif not isinstance (excludedColumns, list):
        print("El argumento excludedColumns debe ser una lista con el nombre de las columnas a excluir.")

    else:
        
        data_copy = data.drop(excludedColumns, axis = 1)
        numerical_columns = [col for col in data.columns if col not in excludedColumns]
        columns_to_standarize = data[numerical_columns]

        scaler = StandardScaler()
        columns_to_standarize = scaler.fit_transform(columns_to_standarize)
        data[numerical_columns] = columns_to_standarize

        data.to_csv('./datos/' + dataName + '_standarized.csv', index = False)

        print("El nuevo fichero estandarizado se ha guardado en el directorio datos con el nombre " + dataName + ".csv")


Definimos las columnas que no vamos a considerar para la estandarización, en este caso las columnas booleanas y las columnas resultado.

In [120]:
titanic_excluded = ['Sex', 'Alone', 'Is_Married', 'Survived']
pcos_excluded = ['Pregnant(Y/N)', 'Weight gain(Y/N)', 'hair growth(Y/N)', 'Skin darkening (Y/N)', 'Hair loss(Y/N)', 'Pimples(Y/N)', 'Fast food (Y/N)','Reg.Exercise(Y/N)', 'PCOS (Y/N)']

dataStandarization(titanicData, 'titanic', titanic_excluded)
dataStandarization(pcosData, 'pcos', pcos_excluded)

El nuevo fichero estandarizado se ha guardado en el directorio datos con el nombre titanic.csv
El nuevo fichero estandarizado se ha guardado en el directorio datos con el nombre pcos.csv


A continuación dividiremos los ficheros estandarizados en dos, uno para el conjunto de entrenamiento y otro para el conjunto de prueba. Para ello definiremos la función necesaria.

In [121]:
def dataSplit (data, dataName):
    if not isinstance (dataName, str):
        print("El argumento dataName debe ser una cadena de carateres.")
    
    else:
        data_train, data_test = model_selection.train_test_split(data, test_size = 0.3, random_state = 99)
        data_train.to_csv('./datos/' + dataName + '_train.csv', index = False)
        data_test.to_csv('./datos/' + dataName + '_test.csv', index = False)

        print('Las dimensiones originales de los datos de entrada son: ', data.shape)
        print('El conjunto de entrenamiento se ha guardado en el directorio datos con el nombre ' + dataName + '_train.csv, y sus dimensiones son: ', data_train.shape)
        print('El conjunto de pruebas se ha guardado en el directorio datos con el nombre ' + dataName + '_test.csv, y sus dimensiones son: ', data_test.shape)
        return data_train, data_test





Finalmente creamos los dos conjuntos de datos a partir del fichero estandarizado:

In [122]:
titanicTrain, titanicTest = dataSplit(titanicData, 'titanic')
pcosTrain, pcosTest = dataSplit(pcosData, 'pcos')

Las dimensiones originales de los datos de entrada son:  (891, 16)
El conjunto de entrenamiento se ha guardado en el directorio datos con el nombre titanic_train.csv, y sus dimensiones son:  (623, 16)
El conjunto de pruebas se ha guardado en el directorio datos con el nombre titanic_test.csv, y sus dimensiones son:  (268, 16)
Las dimensiones originales de los datos de entrada son:  (541, 40)
El conjunto de entrenamiento se ha guardado en el directorio datos con el nombre pcos_train.csv, y sus dimensiones son:  (378, 40)
El conjunto de pruebas se ha guardado en el directorio datos con el nombre pcos_test.csv, y sus dimensiones son:  (163, 40)


Una vez estandarizados los ficheros de entrenamientos, definiremos una función para que, dado un conjunto de datos, se generen diferentes conjuntos de datos de entrenamiento aplicando primero la técnica de Bootstrapping y más adelante Rndom Subspaces.

In [123]:

def generadorConjutosEntrenamiento(fileName, data, amountFiles):

    res= [] #lista de conjuntos de entrenamientos

    col = data[data.columns[:-1]]# cogemos todas las columnas salvo la ultima 
    
    objectiveVariable = data.loc[:, data.columns == data.columns[-1]] #guardamos la ultima columna

    numCol=col.shape[1] #dimension de la columna en col

    #hacemos bootstrapping
    for i in range (amountFiles):
        bootstrap_sample = data.sample(frac = 0.8, replace = True) 
        #para cada subset consideramos solo la mitad de los datos de entrada y con posibilidad de repeticion
        for j in range (amountFiles): #para cada nuevo archivo bootstrappeado
            #cogemos una serie de columnas aleatorias    
            selcted_column=np.random.choice(numCol,size=int(np.sqrt(numCol)),replace=False)#seleccionamos un subconjunto de columnas 
            subspace_sample = col.iloc[:, selcted_column].copy()#creamos un nuevo DataSet 
            #-----------------------------A LO MEJOR HAY QUE CAMBIARLO---------------
            subspace_sample[data.columns.values[-1]] = objectiveVariable #añadimos columna survived
            #------------------------------------------------------------------------

            res.append(subspace_sample)

            route = f'./datos/conjuntosEntrenamiento/{fileName}_trainSet_{i+1}.{j+1}.csv'
            subspace_sample.to_csv(route, index = False)
            print('Fichero creados en la ruta: ', route)

    return res

A continuación definiremos la función encargada del entrenamiento de modelos, la cual llamará a la función generadora definida previamente. Para ello definiremos previamente una función que nos será útil para dividir las variables objetivos del resto de variables.

También definiremos dos funciones, una que dada una lista de listas de valores, nos devuelva una lista con las modas de los valores de la misma posición, y otra que dado una lista de modelos entrenados y un conjuto de datos, nos prediga el valor de la variable objetivo utilizando la moda de los resultdos obtenidos en cada uno de los modelos.

A continuación definiremos las siguientes funciones:
- separarVariables: Útil para dividir las variables objetivos del resto de variables.
- entrenamientoDeModelos: Algoritmo encargado de entrenar una serie de modelos a partir de un conjunto de entrenamiento, para ello necesitara diferentes argumentos:
    - data: Conjunto de datos de entrenamiento.
    - numModelos: Numero de modelos a entrenar, debido a als técnicas aplicadas, el resultado final de modelos es igual a numModelos * numModelos.
    - algoritmo: Argumento de tipo String, de valor "TREE" o "SGD" en función del algoritmo deseado para el entrenamiento de los modelos.
    - proporcionColumnas: Numero entre 0 y 1 que representa el porcentaje de columnas empleadas para el entrenamiento de los modelos.
    - fileName: String que representa el nombre del archivo, para seguir la coherencia del proyecto, esta será "titanic" o "pcos" en función del conjunto de datos.
- modasLista: Dada una lista de listas de numeros, devuelve una lista con las modas de los numeros de una misma posición.
- algoritmoPrediccion: A partir de unos datos de pruebas y una lista de modelos, predice el valor de la variable objetivo.

In [124]:
def separarVariables(data):
    x = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    return x, y


def entrenamientoDeModelos(data, numModelos, algoritmo, proporcionColumnas, fileName):
    res = []
# si no esta entre 0 y 1 paro
    if not 0 <= proporcionColumnas <= 1:
        print("El parametro proporcionColumnas debe ser un numero entre 0 y 1")

    else:
    #si el argumento no es un string
        if not isinstance(algoritmo, str): 
            print("El argumento algoritmo no es un String")
    #si es un string
        else:
            #dado el dataset original obtengo una lista de datos de entrenamiento (hay que ver el numModelos)
            training_data = generadorConjutosEntrenamiento(fileName, data, numModelos)
#por cada dato
            for i in training_data:
                #vego que algoritmo se solicita y creo una nueva instancia del mismo
                if algoritmo.upper() == 'TREE':
                    alg = DecisionTreeClassifier()
        
                elif  algoritmo.upper() == 'SGD':
                    alg = SGDClassifier()
    #separo las variables de datos de las objetivos y ajusto entreno el algoritmo
                x, y = separarVariables(i)
                alg.fit(x, y)
#meto en una lista el modelo entrenado y las columnas empleadas en su entrenamiento
                res.append((alg,x.columns))
    return res

def modasLista(list_of_lists):
    result = []
    list_length = len(list_of_lists[0])

    for i in range(list_length):
        elements = [lst[i] for lst in list_of_lists]    
        result.append(mode(elements))

    return result

def algoritmoPrediccion(datosTesteo, conjunto):
    ls = []
    for i in conjunto:
        pred = i[0].predict(datosTesteo[i[1]])
        ls.append(pred)
    return  modasLista(ls)

Finalmente predecimos los resultados al pasarle un conjunto de datos de pruebas:

In [128]:
titanicModels = entrenamientoDeModelos(titanicTrain, 3, 'sgd', 0.5, 'titanic')
pcosModels = entrenamientoDeModelos(pcosTrain, 4, 'tree', 0.5, 'pcos')

titanicPredictions = algoritmoPrediccion(titanicTest, titanicModels)
pcosPredictions = algoritmoPrediccion(pcosTest, pcosModels)

Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_1.1.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_1.2.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_1.3.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_2.1.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_2.2.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_2.3.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_3.1.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_3.2.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_3.3.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_1.1.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_1.2.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainS

In [129]:

def balancedAccuracyScore(testingData, predicted):
    return balanced_accuracy_score(testingData.iloc[:,-1].tolist(), predicted)

def f1Score(testingData, predicted):
    return f1_score(testingData.iloc[:,-1].tolist(), predicted)


In [131]:
print(balancedAccuracyScore(titanicTest, titanicPredictions))
print(balancedAccuracyScore(pcosTest, pcosPredictions))

print(f1Score(titanicTest, titanicPredictions))
print(f1Score(pcosTest, pcosPredictions))

0.7300684764000979
0.7311957569913211
0.6486486486486486
0.6382978723404255
